In [1]:
import yfinance as yf
import pandas as pd
from ta import add_all_ta_features
from ta.trend import SMAIndicator, EMAIndicator
from ta.momentum import RSIIndicator, StochasticOscillator, StochRSIIndicator, WilliamsRIndicator, ROCIndicator
from ta.trend import ADXIndicator, CCIIndicator, MACD, macd
from ta.volatility import AverageTrueRange, BollingerBands
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Fetch the data from Yahoo Finance and return it as a dataframe
def fetch_ohlcv_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data.reset_index(inplace=True)
    return data

In [3]:
ticker = 'IOO.AX'  # SharesGlobal 100 ETF (IOO) listed on the Australian market
start_date = '2021-01-01'
end_date = '2023-05-06'

# Fetch the data
data = fetch_ohlcv_data(ticker, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [4]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,83.000000,83.000000,81.550003,81.839996,81.839996,27405
1,2021-01-05,81.690002,81.690002,80.949997,81.040001,81.040001,28391
2,2021-01-06,81.099998,81.330002,80.400002,80.449997,80.449997,37254
3,2021-01-07,80.809998,81.800003,80.629997,81.070000,81.070000,42802
4,2021-01-08,81.510002,82.470001,81.510002,82.279999,82.279999,8752
...,...,...,...,...,...,...,...
585,2023-05-01,109.360001,109.360001,108.940002,109.010002,109.010002,31654
586,2023-05-02,108.980003,108.980003,107.389999,107.389999,107.389999,31082
587,2023-05-03,107.389999,107.680000,107.320000,107.680000,107.680000,35506
588,2023-05-04,107.680000,107.680000,106.760002,106.900002,106.900002,35800


In [5]:
# SMA: average "column_name" prices over the past "window" periods
# Buy: when it lower than the current price
# Sell: when it higher than the current price
def add_sma(data, column_name, window):
    sma_indicator = SMAIndicator(data[column_name], window=window)
    data[f'sma_{window}'] = sma_indicator.sma_indicator()
    return data

# EMA: average "column_name" prices over the past "window" periods but recent price have a higher weight
# Buy: when it lower than the current price
# Sell: when it higher than the current price
def add_ema(data, column_name, window):
    ema_indicator = EMAIndicator(data[column_name], window=window)
    data[f'ema_{window}'] = ema_indicator.ema_indicator()
    return data

# RSI: measures the speed of price changes. RSI=100-100/(1+RS),
#      where RS is the ratio of the average up close to the average down close over the past "window" periods
# Return: The return value of this function is the RSI value, which is a number between 0 and 100
# Buy: when the RSI is less than "oversold"(generally is 30), the market is considered oversold
# Sell: when RSI is greater than "overbought"(generally is 70), the market is considered overbought
# Neutral: when RSI is between "oversold" and "overbought"
def add_rsi(data, column_name, window):
    rsi_indicator = RSIIndicator(data[column_name], window=window)
    data[f'rsi_({window})'] = rsi_indicator.rsi()
    return data

# STOCH: measure the momentum of an asset's price over a "k_window" period of time
#        First, %K = (C-L)/(H-L)*100 (represents the current price in relation to recent price range)
#        where C is the most recent closing price, L and H is the lowest/highest price during "k_window" period
#        Then, %D represents the "d_window"-period average of %K.
#        STOCH(9,6) is more sensitive, but STOCH(14,3,3) is popular(the last '3' means 3-period average of %D)
# Return: The return value of this function is stoch(), which means the value of %K
# Threshold: The default thresholds are 20 for oversold and 80 for overbought.
def add_stoch(data, high_column, low_column, close_column, k_window, d_window):
    stoch_indicator = StochasticOscillator(data[high_column], data[low_column], 
                                           data[close_column], k_window, d_window)
    data[f'stoch_({k_window},{d_window})'] = stoch_indicator.stoch_signal()
    return data

# StochRSI: measure the momentum of RSI over a "window" period of time
#           First, RSI = 100-100/(1+RS), where RS is as we said above over the past "window" periods
#           Next, StochRSI = (RSI-L)/(H-L)*100 (represents the current RSI in relation to recent RSI range)
#           where L and H is the lowest/highest RSI during "window" period
#           Then, %K is the moving average of StochRSI over "k_window" period
#           Finally, %D is the moving average of %K over "d_window" period
#           STOCHRSI(14,14,3,3) is popular, which means 14-period RSI, 14-period StochRSI, 3-period %K, 3-period %D
# Return: The return value of this function is stochrsi(), which means the StochRSI value here
# Overbought: when the StochRSI is greater than "overbought", default is 80
# Oversold: when the StochRSI is less than "oversold", default is 20
#           But here it just simply alert traders that RSI is near the extremes of its recent readings
def add_stochrsi(data, column_name, window, k_window, d_window, overbought=80, oversold=20):
    stochrsi_indicator = StochRSIIndicator(data[column_name], window, k_window, d_window)
    data[f'stochrsi_({window},{window},{k_window},{d_window})'] = stochrsi_indicator.stochrsi()
    return data

# MACD: Moving Average Convergence Divergence, shows the relation between two moving averages of prices
#       The MACD line is calculated by EMA(12)-EMA(26), 12 and 26 are "short_window" and "long_window" here
#       Then, a "signal_window"-day EMA of the MACD line is called the signal line, which is plotted on top of the MACD line
# Return: The return value of this function is macd(), which means the MACD line value here
# Buy: when the MACD line crosses above the signal line
# Sell: when the MACD line crosses below the signal line
def add_MACD(data, column_name, short_window, long_window, signal_window):
    macd_indicator = MACD(data[column_name], short_window, long_window, signal_window)
    data[f'macd_({short_window},{long_window},{signal_window})'] = macd_indicator.macd()
    data[f'macd_diff_({short_window},{long_window},{signal_window})'] = macd_indicator.macd_diff()
    data[f'macd_signal_({short_window},{long_window},{signal_window})'] = macd_indicator.macd_signal()
    return data
def add_macd(data, column_name, window_fast=12, window_slow=26):
    macd_indicator = macd(data[column_name], window_slow, window_fast)
    data[f'macd_({window_fast},{window_slow})'] = macd_indicator
    return data

# ADX: Average Directional Movement Index, measures the strength of a trend
#      ADX = 100 * EMA(ABS(+DI-(-DI))/(+DI+(-DI)), window)
#      where +DI and -DI are the positive and negative directional indicators
#      +DI = 100 * EMA(+DM/TR, window), -DI = 100 * EMA(-DM/TR, window)
#      where +DM = +DM = H - H(-1), -DM = L(-1) - L
#      where H is the current high, H(-1) is the previous high, L(-1) is the previous low, L is the current low
#      TR = MAX(H-L, ABS(H-C(-1)), ABS(L-C(-1)))
#      where C(-1) is the previous close, C is the current close
# Return: The return value of this function is adx(), which means the ADX value here
# Buy: a strong trend is present when ADX is above 20 (or 25)
# Sell: no trend is present when ADX is below 20 (or 25)
def add_adx(data, high_column, low_column, close_column, window):
    adx_indicator = ADXIndicator(data[high_column], data[low_column], data[close_column], window)
    data[f'adx_({window})'] = adx_indicator.adx()
    return data

# Williams %R: a momentum indicator that is the inverse of the Fast Stochastic Oscillator
#              %R = (Highest High - Close)/(Highest High - Lowest Low) * -100
#              the %R range is from 0 to -100, usually use -20 as the overbought threshold and -80 as the oversold threshold
# Return: The return value of this function is williams_r(), which means the Williams %R value here
# Oversold: From -100 to -80, this indicates oversold market condition
# Overbought: From 0 to -20, this indicates overbought market condition
# Neutral: From -80 to -20, this indicates normal market condition
def add_williamsr(data, high_column, low_column, close_column, window):
    williamsr_indicator = WilliamsRIndicator(data[high_column], data[low_column], data[close_column], window)
    data[f'williamsr_({window})'] = williamsr_indicator.williams_r()
    return data

# CCI: measures the difference between price change and its average price change to identify new trend or extreme conditions
#      Assume the "window" is 20, so CCI = (Typical Price - 20-period SMA of TP) / (0.015 * Mean Deviation)
#      where TP = (H+L+C)/3, Mean Deviation = Sum(|TP - 20-period SMA of TP|)/(20)
# Return: The return value of this function is cci(), which means the CCI value here
# Buy: When the CCI moves above +100, a new, strong uptrend is beginning, signaling a buy
# Sell: When the CCI moves below −100, a new, strong downtrend is beginning, signaling a sell
# Neutral: Between +100 and −100, this indicates normal market condition
def add_cci(data, high_column, low_column, close_column, window):
    cci_indicator = CCIIndicator(data[high_column], data[low_column], data[close_column], window)
    data[f'cci_({window})'] = cci_indicator.cci()
    return data

# ATR: Average True Range, measures market volatility over "window" days (usually 14 days)
#      ATR = EMA(TR, window), where TR = MAX(H-L, ABS(H-C(-1)), ABS(L-C(-1)))
#      where H is the current high, H(-1) is the previous high, L(-1) is the previous low, L is the current low
#      where C(-1) is the previous close, C is the current close
#      TR is the true range, which measures the greatest of the following:
#      1. Current high less the current low
#      2. The absolute value of the current high less the previous close
#      3. The absolute value of the current low less the previous close
# Return: The return value of this function is average_true_range(), which means the ATR value here
# Less Volatility: A low ATR value indicates less volatility
# High Volatility: A high ATR value indicates high volatility
#                 However, no threshold here, it depends on the context and the timeframe being analyzed
def add_atr(data, high_column, low_column, close_column, window):
    atr_indicator = AverageTrueRange(data[high_column], data[low_column], data[close_column], window)
    data[f'atr_({window})'] = atr_indicator.average_true_range()
    return data


# ROC: Rate of Change, measures the percentage change in price between the current price and the price "window" days ago
#      ROC = (Close - Close(-window))/Close(-window)
#      where Close is the current close, Close(-window) is the close "window" days ago
# Return: The return value of this function is rate_of_change(), which means the ROC value here
# Buy: When the ROC crosses above the zero line, this indicates a buy signal
# Sell: When the ROC crosses below the zero line, this indicates a sell signal
def add_roc(data, column_name, window):
    roc_indicator = ROCIndicator(data[column_name], window)
    data[f'roc_({window})'] = roc_indicator.roc()
    return data

# Bull bear power: measures the ability of bulls and bears to push price beyond an SMA of price
#                  Bull Power = High - EMA(Close, window)
#                  Bear Power = Low - EMA(Close, window)
# Return: The return value of this function is bull_bear_power(), which means the bull bear power value here
# Buy: When the Bull Power crosses above zero, this indicates a buy signal
# Sell: When the Bear Power crosses below zero, this indicates a sell signal
def add_bull_bear_power(data, high_column, low_column, close_column, window=13):
    ema = data[close_column].ewm(span=window).mean()
    bull_power = data[high_column] - ema
    bear_power = data[low_column] - ema
    data[f'bull/bear power_({window})'] = bull_power + bear_power
    return data

In [6]:
# Add SMA and EMA for the close price with certain window
window_list = [5, 10, 20, 50]
for window_size in window_list:
    data = add_sma(data, 'Close', window_size)
    data = add_ema(data, 'Close', window_size)

In [7]:
data = add_rsi(data, 'Close', 14)
data = add_stoch(data, 'High', 'Low', 'Close', 9, 6)
data = add_stochrsi(data, 'Close', 14, 3, 3)
data = add_macd(data, 'Close', 12, 26)
# data = add_MACD(data, 'Close', 12, 26, 9)
data = add_adx(data, 'High', 'Low', 'Close', 14)
data = add_williamsr(data, 'High', 'Low', 'Close', 14)
data = add_cci(data, 'High', 'Low', 'Close', 14)
data = add_atr(data, 'High', 'Low', 'Close', 14)
data = add_roc(data, 'Close', 14)
data = add_bull_bear_power(data, 'High', 'Low', 'Close', 13)
# Add all ta features
# data = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume")

In [8]:
data.tail(5)

,Date,Open,High,Low,Close,Adj Close,Volume,sma_5,ema_5,sma_10,...,rsi_(14),"stoch_(9,6)","stochrsi_(14,14,3,3)","macd_(12,26)",adx_(14),williamsr_(14),cci_(14),atr_(14),roc_(14),bull/bear power_(13)
585,2023-05-01,109.360001,109.360001,108.940002,109.010002,109.010002,31654,107.992001,108.105126,106.981001,...,73.954355,90.048991,0.761581,1.962883,31.627849,-6.809310,164.376624,0.927716,4.525843,5.019914
586,2023-05-02,108.980003,108.980003,107.389999,107.389999,107.389999,31082,108.178001,107.866750,107.098000,...,62.620950,84.472893,0.000000,1.864388,31.350915,-38.326877,75.991719,0.977165,2.004180,2.875641
587,2023-05-03,107.389999,107.680000,107.320000,107.680000,107.680000,35506,108.104001,107.804500,107.302000,...,63.693576,78.470627,0.079967,1.789107,31.024728,-32.684835,45.808239,0.933082,2.591465,1.239118
588,2023-05-04,107.680000,107.680000,106.760002,106.900002,106.900002,35800,108.028001,107.503001,107.417001,...,58.805714,67.681166,0.000000,1.647516,30.167701,-54.304630,10.368336,0.932147,2.247727,0.673532
589,2023-05-05,106.800003,106.800003,106.080002,106.250000,106.250000,34866,107.446001,107.085334,107.422001,...,55.016726,55.863839,0.000000,1.465955,28.741989,-81.842056,-48.291144,0.924137,1.113436,-0.705541


In [9]:
def parse_column_name(column_name):
    indicator_type, params_str = column_name.split("_", 1)
    params = [int(x) for x in re.findall(r'\d+', params_str)]
    return indicator_type, params

def get_signal(data, column_name):
    indicator_type, params = parse_column_name(column_name)
    value = data[column_name].iloc[-1]
    signal = "Neutral"

    if indicator_type == "sma" or indicator_type == "ema":
        current_price = data["Close"].iloc[-1]
        if value < current_price:
            signal = "Buy"
        elif value > current_price:
            signal = "Sell"

    elif indicator_type == "rsi":
        oversold = 30
        overbought = 70
        if value < oversold:
            signal = "Oversold"
        elif value > overbought:
            signal = "Overbought"

    elif indicator_type == "stoch":
        oversold = 20
        overbought = 80
        if value < oversold:
            signal = "Oversold"
        elif value > overbought:
            signal = "Overbought"

    elif indicator_type == "stochrsi":
        oversold = 20
        overbought = 80
        if value < oversold:
            signal = "Oversold"
        elif value > overbought:
            signal = "Overbought"

    elif indicator_type == "macd":
        if value > 0:
            signal = "Buy"
        elif value < 0:
            signal = "Sell"

    elif indicator_type == "adx":
        strong_trend = 25
        if value > strong_trend:
            signal = "Buy"
        else:
            signal = "Sell"

    elif indicator_type == "williamsr":
        overbought = -20
        oversold = -80
        if value > overbought:
            signal = "Overbought"
        elif value < oversold:
            signal = "Oversold"

    elif indicator_type == "cci":
        overbought = 100
        oversold = -100
        if value > overbought:
            signal = "Buy"
        elif value < oversold:
            signal = "Sell"

    elif indicator_type == "atr":
        threshold = 0.06
        if value >= threshold:
            signal = "Less Volatility"
        elif value < threshold:
            signal = "High Volatility"

    elif indicator_type == "roc":
        if value > 0:
            signal = "Buy"
        elif value < 0:
            signal = "Sell"

    elif indicator_type == "bull/bear power":
        if value > 0:
            signal = "Buy"
        elif value < 0:
            signal = "Sell"

    return signal

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  590 non-null    datetime64[ns]
 1   Open                  590 non-null    float64       
 2   High                  590 non-null    float64       
 3   Low                   590 non-null    float64       
 4   Close                 590 non-null    float64       
 5   Adj Close             590 non-null    float64       
 6   Volume                590 non-null    int64         
 7   sma_5                 586 non-null    float64       
 8   ema_5                 586 non-null    float64       
 9   sma_10                581 non-null    float64       
 10  ema_10                581 non-null    float64       
 11  sma_20                571 non-null    float64       
 12  ema_20                571 non-null    float64       
 13  sma_50              

In [11]:
column_name = "bull/bear power_(13)"
signal = get_signal(data, column_name)
print(signal)

bull/bear power [13] -0.7055413561469379
Sell


In [12]:
haha

NameError: name 'haha' is not defined

In [ ]:
def count_signals(data, signal_type, window_list):
    signal_columns = [f'{indicator}_{window_size}_signal' for indicator in ['sma', 'ema'] for window_size in window_list]
    return data[signal_columns].apply(lambda x: (x == signal_type).sum(), axis=1)

In [ ]:
window_list = [5, 10, 20, 50]
data_with_signals = add_signals(data, 'Close', window_list)
data_with_signals['Buy_count'] = count_signals(data_with_signals, 'Buy', window_list)
data_with_signals['Sell_count'] = count_signals(data_with_signals, 'Sell', window_list)

In [ ]:
data_with_signals[-5:].loc[:, ['Date','Close','Volume',
                               'sma_5','ema_5','sma_10','ema_10','sma_20','ema_20',
                               'sma_50','ema_50','sma_100','ema_100']]

In [ ]:
def generate_summary_table(data, current_price, window_list):
    columns = ['Period', 'Simple', 'Exponential']
    summary_table = pd.DataFrame(columns=columns)

    for window_size in window_list:
        sma_signal = get_signal({'Close': current_price,
                                 'sma_{}'.format(window_size): data['sma_{}'.format(window_size)].iloc[-1]}, 
                                'Close', 'sma_{}'.format(window_size))
        ema_signal = get_signal({'Close': current_price, 
                                 'ema_{}'.format(window_size): data['ema_{}'.format(window_size)].iloc[-1]}, 
                                'Close', 'ema_{}'.format(window_size))
        summary_table = summary_table.append(pd.Series([f'MA{window_size}', 
                                                        f'{data["sma_{}".format(window_size)].iloc[-1]:.2f} {sma_signal}', 
                                                        f'{data["ema_{}".format(window_size)].iloc[-1]:.2f} {ema_signal}'], 
                                                       index=columns), ignore_index=True)

    buy_count = (summary_table['Simple'].apply(lambda x: x.split()[1]) == 'Buy').sum() + (summary_table['Exponential'].apply(lambda x: x.split()[1]) == 'Buy').sum()
    sell_count = (summary_table['Simple'].apply(lambda x: x.split()[1]) == 'Sell').sum() + (summary_table['Exponential'].apply(lambda x: x.split()[1]) == 'Sell').sum()
    summary = 'BUY' if buy_count > sell_count else 'SELL'

    summary_table = summary_table.append(pd.Series(['Buy:', buy_count, f'Sell: {sell_count}'], index=columns), ignore_index=True)
    summary_table = summary_table.append(pd.Series(['Summary:', '', summary], index=columns), ignore_index=True)

    return summary_table

current_price = 106.92
summary_table = generate_summary_table(data_with_signals, current_price, window_list)
print(summary_table)

In [ ]:
summary_table